# Diving Into Computer Vision
*Curtis Miller*

In this notebook I demonstrate how to use some tools for basic image manipulation; specifically, we will see how to use the Python Image Library (PIL), how to see images using matplotlib, how to create NumPy arrays from images, and using SciPy with images.

## PIL and Matplotlib

Here I work with PNG format images. Below I show how to load in an image with PIL and view it with matplotlib.

In [ ]:
import PIL
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import copy
from scipy.interpolate import griddata
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (18, 16)

In [ ]:
house = PIL.Image.open("house.png")
plt.imshow(house)

If we want to see the image in grayscale, we will need to convert it. Below converts to grayscale.

In [ ]:
plt.imshow(house.convert('L'))    # Converts to grayscale, which is not how it appears in plot

In [ ]:
plt.imshow(house.convert('L'), cmap="hot")

In [ ]:
plt.imshow(house.convert('L'), cmap="gray")

The `rotate()` method rotates an image.

In [ ]:
plt.imshow(house.rotate(45))

And here is resizing.

In [ ]:
plt.imshow(house.resize((16, 16)))

We could crop an image to create a new sub-image, passing a tuple with the coordinates of the top-left and bottom-right corners.

In [ ]:
plt.imshow(house.crop((200, 200, 500, 500)))

Then we could transpose the image and even paste it in another image.

In [ ]:
plt.imshow(house.crop((200, 200, 500, 500)).transpose(PIL.Image.ROTATE_180))

In [ ]:
box = (200, 200, 500, 500)     # First two: coordinates of upper left corner; last two: coordinates of lower right
house2 = copy.deepcopy(house)
house2.paste(house2.crop(box).transpose(PIL.Image.ROTATE_180), box)
plt.imshow(house2)

## NumPy and Images

We can create a NumPy array that holds the RGB values of an image like so:

In [ ]:
house_arr = np.array(house.convert("RGB"))
house_arr.shape

In [ ]:
house_arr[:5, :5, :3]

In [ ]:
house_arr[:5, :5, 0]    # Red

In [ ]:
plt.imshow(255 - house_arr[:, :, 0], cmap="Reds")

In [ ]:
plt.imshow(255 - house_arr[:, :, 1], cmap="Greens")

In [ ]:
plt.imshow(house_arr[:, :, 2], cmap="Blues")

We could create a histogram to see how many pixels have particular RGB intensities.

In [ ]:
plt.hist(house_arr[:, :, 0].flatten(), bins=256)[2]

In [ ]:
plt.hist(house_arr[:, :, 1].flatten(), bins=256)[2]

In [ ]:
plt.hist(house_arr[:, :, 2].flatten(), bins=256)[2]

## SciPy

Now that we can turn images into NumPy arrays, we can use scipy to work with them as well. The code below is for interpolation when rescaling images. (I first introduced this code in my video course, [*Training Your Systems with Python Statistical Modeling*](https://www.packtpub.com/big-data-and-business-intelligence/training-your-systems-python-statistical-modeling-video).)

In [ ]:
def griddata_point_format(arr, normalize=True):
    """Converts grayscale image stored in ndarray arr into a format acceptable for griddata, returning a dict
       If normalize is True, coordinates are on a 0-1 scale"""
    
    shape = arr.shape
    x = np.arange(shape[0])
    y = np.arange(shape[1])
    coord_mat = np.transpose([np.tile(x, shape[1]), np.repeat(y, shape[0])])    # Construct a matrix of coordinates
    values = arr[coord_mat[:, 0], coord_mat[:, 1]]    # Construct a 1D array containing the intensity values of the image
                                                      # at the given coordinates
    if normalize:
        # All coordinates will be between 0 and 1
        coord_mat = np.array(coord_mat, dtype=np.float64)
        coord_mat[:, 0] = coord_mat[:, 0] / shape[0]    # Note that 1 is not actually attained; that's fine
        coord_mat[:, 1] = coord_mat[:, 1] / shape[1]
        
    return {"points": coord_mat, "values": values}

def interp_points(length, width, normalize=True):
    """Gets two NumPy arrays corresponding to the points where interpolation should occur"""
    
    grid_x, grid_y = np.mgrid[0:length, 0:width]
    if normalize:
        # This option should be used if normalize is True in griddata_point_format
        # All points will be between 0 and 1
        grid_x = np.array(grid_x, dtype=np.float64)
        grid_y = np.array(grid_y, dtype=np.float64)
        grid_x = grid_x / length
        grid_y = grid_y / width
    
    return (grid_x, grid_y)

In [ ]:
im0 = griddata(xi = interp_points(1024, 1024), **griddata_point_format(house_arr[:128, :128, 0]), method='cubic')
im0

In [ ]:
plt.imshow(255 - house_arr[:128, :128, 0], cmap="Reds")

In [ ]:
plt.imshow(255 - im0, cmap="Reds")

There's a lot more we can do with these packages; we will see more later.